In [1]:
!pip install transformers torch stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
from transformers import pipeline


generator = pipeline('text-generation', model='gpt2')

def generate_text(prompt, max_length=150):
    response = generator(prompt, max_length=max_length, num_return_sequences=1)
    return response[0]['generated_text']

prompt = "This is sunny day, and"
print(generate_text(prompt))

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


This is sunny day, and wind is blowing in to the west of us, but to stay and drive I had to drop off the bike. I knew then that we weren't going to need the wheels, and that it would be safer than going out and doing the road in all directions without getting tangled."

While the city's approach has worked out fine, the car has been one of the most difficult rides by his two brothers and the other's older sister. "I don't know why the father was on the steering wheel, but he's always telling me that it would be better if he wouldn't be out there at that exact moment," he says. "That's the biggest thing that gets me mad: I need him out


In [3]:
!pip install 'shimmy>=2.0'

In [4]:
from os import truncate
import gymnasium as gym
import numpy as np
import torch
from stable_baselines3 import PPO


class ContentFeedbackEnv(gym.Env):
  def __init__(self):
    super(ContentFeedbackEnv, self).__init__()
    self.action_space = gym.spaces.Discrete(3) # 0: 싫어요, 1: 좋아요, 2: 유해 컨텐츠 보고
    self.observation_space = gym.spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)
    self.history = []

  def step(self, action):
    if action == 1:
      reward = 1
      feedback = "Like"
    elif action == 2:
      reward = -2
      feedback = "Danger"
    else:
      reward = -1
      feedback = "Hate"

    self.history.append(feedback)

    obs = np.array([0.5])
    terminated = False
    truncate = False
    info = {}

    return obs, reward, terminated, truncate, info

  def reset(self, seed=None, options=None):
    super().reset(seed=seed)
    return np.array([0.5]), {}

In [5]:
env = ContentFeedbackEnv()
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [6]:
past_feedback = [1, 0, 2, 1, 1, 0, 2, 1, 0, 1]

for action in past_feedback:
  obs, reward, terminated, truncate, info = env.step(action)
  print(f"Action: {action}, Observation: {obs}, Reward: {reward}")

model.learn(total_timesteps=10000)
model.save('rlhf_content_model')

Action: 1, Observation: [0.5], Reward: 1
Action: 0, Observation: [0.5], Reward: -1
Action: 2, Observation: [0.5], Reward: -2
Action: 1, Observation: [0.5], Reward: 1
Action: 1, Observation: [0.5], Reward: 1
Action: 0, Observation: [0.5], Reward: -1
Action: 2, Observation: [0.5], Reward: -2
Action: 1, Observation: [0.5], Reward: 1
Action: 0, Observation: [0.5], Reward: -1
Action: 1, Observation: [0.5], Reward: 1
-----------------------------
| time/              |      |
|    fps             | 603  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 300          |
|    iterations           | 2            |
|    time_elapsed         | 13           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0138440095 |
|    clip_fraction        | 0.3

In [9]:
model = PPO.load('rlhf_content_model')

env = ContentFeedbackEnv()
model.set_env(env)

prompt = "This is rainy day, so"
response = generate_text(prompt)

print(response)

action = 1

env.step(action)
model.learn(total_timesteps=10)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
This is rainy day, so I was wondering how I could get a lot of rain for my weekend party in January and start to shower a little. I bought a 1.5 gallon bucket for both parties.

So I did:


I made 6 of these and were able to get the little hose running along the back and the front with only a small problem that they got stuck onto the bottom. It is really annoying. I ran two 8-12 inch deep water dousing water over a water pump. On the third time this happened I ended up blowing a stream at the bottom of my hose and that had to be done in about 5 minutes. I was trying to clean it out in 3 minute increments.


I ended
-----------------------------
| time/              |      |
|    fps             | 1032 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
